# KAKENの研究機関マスタをローカルのMariaDBに保存するプログラム

### 事前準備
- KAKENマスタデータは、git のリポジトリで管理されており、最新のデータを利用可能。
- https://bitbucket.org/niijp/grants_masterxml_kaken/ からリポジトリを pull して、ローカルの ./grants_masterxml_kaken フォルダに同期しておく。
- ローカルで MariaDB を動かしておく。
- MariaDB のユーザ名、パスワード、データベース名は、configparserで ./config.ini として保存しておく。

### ここから本編
bitbucketから読み込んだマスタのXMLファイルをelementTreeに変換

In [ ]:
# encoding: utf-8
from lxml import etree
import pandas as pd

In [ ]:
tree = etree.parse('grants_masterxml_kaken/institution_master_kakenhi.xml')

institutionlist = []
for institution_table in tree.iterfind("institution_table"):
    for institution in institution_table.iterfind("institution"):
        institution_name = institution.find("name[@lang='ja']").text
        institution_niicode = institution.find("code[@type='nii']").text

        row = [
            institution_niicode,
            institution_name,
        ]
        
        institutionlist.append(row)
            
df = pd.DataFrame(institutionlist)
df.columns = ['institution_niicode', 'institution_name']
df

重複のデータがあるかどうか

In [ ]:
df.duplicated().any()

重複データがあった。dropしておく。

In [ ]:
df = df.drop_duplicates()
df.duplicated().any()

In [ ]:
df.institution_niicode.value_counts()

In [ ]:
df = df.astype({
    'institution_niicode': int,
})
df.info()

niicodeがユニークなので、インデックスに設定する

In [ ]:
df = df.set_index('institution_niicode')
df

ローカルのmariaDBに関する設定ファイルを読み込み（config.iniはgitに上げていないが、事前準備で作られているはず）

In [ ]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')
username = config['mariadb']['username']
password = config['mariadb']['password']
database = config['mariadb']['database']
url = 'mysql+pymysql://' + username + ':' + password + '@localhost:3306/' + database + '?charset=utf8'

データベースにテーブル構造とデータの中身を書き込む

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.types import String, Integer

engine = create_engine(url, echo=True)

df.to_sql('kaken_master_institution', engine, if_exists='replace',
          dtype={
              'institution_niicode': Integer,
              'institution_name': String(256),
                })

### おしまい
データがコミットされていれば終了。HeidiSQLなどで、上記ドロップ済みデータフレームの件数が登録されているか確認する。